# Pipeline Chatbot PONAL - Bedrock

In [19]:
import os
import boto3
import json
import time
import pandas as pd

In [2]:
print(boto3.__version__)

1.37.25


In [3]:
os.environ["AWS_PROFILE"] = "PONAL" 
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

## Obtención Query

### Obtención Esquema

In [17]:
# Configurar el cliente de AWS Glue
glue_client = boto3.client('glue', region_name='us-east-1')

# Parámetros: base de datos y nombre de la tabla en Glue
database_name = "pon_prod_siedco_raw"
table_name = "raw__cnp_usr4dblsiedco2aws_delitos_siedco"

# Obtener metadatos de la tabla
response = glue_client.get_table(DatabaseName=database_name, Name=table_name)
schema = response['Table']['StorageDescriptor']['Columns']
schema_str = "\n".join([f"{col['Name']} - {col['Type']}" for col in schema])
schema_str = f"\nTable Name: {table_name}\nDatabase Name: {database_name}" + schema_str

# Mostrar el esquema
print(f"{schema_str}")


Table Name: raw__cnp_usr4dblsiedco2aws_delitos_siedco
Database Name: pon_prod_siedco_rawid_consecutivo - decimal(38,0)
id_delito - decimal(38,0)
delito - string
direccion - string
id_municipio - decimal(38,0)
municipio - string
fecha_hecho - timestamp
hora - timestamp
fecha_actualiza - timestamp
departamento - string
id_hecho - decimal(38,0)
longitud - string
id_departamento - decimal(38,0)
latitud - string


### Consulta a Bedrock

In [ ]:
# Crear el cliente de Amazon Bedrock Runtime
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # Reemplaza con tu región de AWS
)
# Especificar el ID del modelo Claude 3.5 Sonnet
model_id = 'anthropic.claude-3-5-sonnet-20240620-v1:0'

# Definir prompt
prompt_init = '''
Eres un chatbot encargado de responder preguntas sobre informacion policial en Colombia. 
A continuacion recibiras una pregunta que realiza el usuario y un esquema de diferentes tablas donde puede habitar la informacion solicitada.

Debes retornar como respuesta una consulta SQL que permita obtener la informacion solicitada por el usuario,
compatible con el servicio de AWS Athena. 
Retorna UNICMAENTE la consulta SQL.
'''
prompt_user = "Cual es el delito mas comun que ocurre en el municipio de ENVIGADO?"

prompt_full = f"Instruciones: {prompt_init}\nPregunta del usuario: {prompt_user}\nEsquema de la tabla:\n{schema_str}"

# Preparar el cuerpo de la solicitud
cuerpo_solicitud = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 300,
    "temperature": 0,
    "messages": [
        {"role": "user", "content": prompt_full}
    ]
})

# Invocar el modelo
respuesta = bedrock_runtime.invoke_model(
    modelId = model_id,
    body = cuerpo_solicitud
)

# Procesar y mostrar la respuesta
respuesta_cuerpo = json.loads(respuesta['body'].read())
query = respuesta_cuerpo['content'][0]['text']
print( f"Respuesta:\n{query}" )

Respuesta:
SELECT delito, COUNT(*) as total
FROM raw__cnp_usr4dblsiedco2aws_delitos_siedco
WHERE municipio = 'ENVIGADO'
GROUP BY delito
ORDER BY total DESC
LIMIT 1;


## Consulta a Athena

In [23]:
import pandas as pd
import time
# Configurar el cliente de Athena
athena_client = boto3.client("athena", region_name="us-east-1")
 
# Parámetros
database_name = "pon_prod_siedco_raw"
output_location = "s3://aws-athena-query-results-926162397524-us-east-1/"  # Bucket donde Athena guarda los resultados

t1 = time.time()
# Ejecutar la consulta en Athena
response = athena_client.start_query_execution(
    QueryString = query,
    QueryExecutionContext = {"Database": database_name},
    ResultConfiguration = {"OutputLocation": output_location},
)
 
# Obtener el ID de la ejecución de la consulta
query_execution_id = response["QueryExecutionId"]
print(f"Consulta en ejecución con ID: {query_execution_id}")
 
# Esperar hasta que la consulta termine
while True:
    status = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    estado = status["QueryExecution"]["Status"]["State"]
    if estado in ["SUCCEEDED", "FAILED", "CANCELLED"]:
        break
t2 = time.time()
print(f"La consulta tomó {t2 - t1:.2f} segundos")

print()

if estado == "SUCCEEDED":
    print('Consulta Exitosa.')
    # Obtener los resultados de la consulta
    resultado = athena_client.get_query_results(QueryExecutionId=query_execution_id)
 
    # Procesar los resultados
    columnas = [col["VarCharValue"] for col in resultado["ResultSet"]["Rows"][0]["Data"]]
    datos = [[col.get("VarCharValue", "NULL") for col in fila["Data"]] for fila in resultado["ResultSet"]["Rows"][1:]]
 
    # Convertir a DataFrame de Pandas
    df = pd.DataFrame(datos, columns=columnas)
else:
    print('Consulta Fallida.')

Consulta en ejecución con ID: abfbba31-bcb8-4498-9032-812c35eceb90
La consulta tomó 2.24 segundos

Consulta Exitosa.


In [28]:
df_string = df.to_string(index=False)
print(df_string)

df

                      delito count
ARTÍCULO 239. HURTO PERSONAS  7384


,delito,count
0,ARTÍCULO 239. HURTO PERSONAS,7384


## Obtencion Respuesta

In [29]:
prompt_init = '''
Eres un chatbot encargado de responder preguntas sobre informacion policial en Colombia. 
A continuacion recibiras una pregunta que realiza el usuario e informacion relevante en formato de Tabla para responder a la pregunta.

Debes retornar una respuesta a la pregunta del usuario basandote en la informacion de la tabla.
'''

prompt_full = f"Instruciones: {prompt_init}\nPregunta del usuario: {prompt_user}\nTabla relevante:\n{df_string}"

cuerpo_solicitud = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 300,
    "temperature": 0,
    "messages": [
        {"role": "user", "content": prompt_full}
    ]
})

respuesta = bedrock_runtime.invoke_model(
    modelId = model_id,
    body = cuerpo_solicitud
)

respuesta_cuerpo = json.loads(respuesta['body'].read())
output = respuesta_cuerpo['content'][0]['text']
print( f"Respuesta:\n{output}" )

Respuesta:
Según la información proporcionada en la tabla, el delito más común que ocurre en el municipio de ENVIGADO es el hurto a personas, específicamente el delito tipificado en el ARTÍCULO 239 del Código Penal colombiano, que se refiere al hurto. Este delito ha sido reportado 7,384 veces, lo que lo convierte en el más frecuente en ese municipio de acuerdo a los datos disponibles.
